In [1]:
# Will reload modeules after this when they change!
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import itertools
import functools
import importlib
import os
from IPython.display import Image

from nl_00_hmdb_pre import main_loop_0
from sklearn.decomposition import PCA
import umap

from matplotlib import pyplot as plt
from scipy.stats import gaussian_kde

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

import warnings

def action_with_warnings():
    warnings.warn("should not appear")

In [3]:
# Regression:
from nl_03_filter_model_score_reg import filter_split_model_score

In [ ]:
# Binary classification:
from nl_03_filter_model_score import filter_split_model_score

### This script runs the neutral loss scripts for MALDI IMS data

Steps:
1. Generation of core metabolome database.  See:
    PycharmProjects/core_metabolome/core_metabolome_db.ipynb
2. Manual curations of core_metabolome_v2 --> core_metabolome_v3

1. nl_00 Calculates Mordred descriptors and FP4 fingerprints as bits.
2. nl_01 Filter HMDB_db for observed and parses METASPACE output.
3. nl_02 Joins HMDB_db and METASPACE output.
4. nl_03 Filters and searchs with direct and machine learning models.

### To do:
5. nl_04 deep learning: executing on toy data, change to regression

### Observations:
1. 2912/3333 datasets a) did not error and b) had at least one id.  
2. 2543/2868 datasets w/10 or more ID's.  Obs: 557,508/559,002
3. 644,094 values identifications, the dataset with the most identifications had 5,820 and mean was 221 (median 131).  Re calculate...
4. The data did not pass the Shapiro-Wilk test for normality, and nonparametrical statistics should be used.

# Off-line steps:
### Generation of core metabolome database.
http://localhost:8888/notebooks/PycharmProjects/core_metabolome/core_metabolome_db.ipynb
### Identification of good quality datasets.
http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/high_quality_data_investigations.ipynb
### Good datasets were researched from the beta server.  Run 1x w/ "reprocess = True" and "reprocess_not_downloading = True".  Run again w/False.
http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/Beta_server_neutral_losses_mass_search.ipynb

In [4]:
### Off-line variables:
core_metabolome = '/Users/dis/PycharmProjects/core_metabolome/core_metabolome_v3.pickle'
good_ds_ids = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/good_ds_2020_Feb_25.txt'
beta_variables = {'NEUTRAL_LOSSES': ['-H2O'], 'MAX_FDR': 0.5, 'molDBs': 'core_metabolome_v3'}
path_to_reports = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/good_nl_reports_core_metabolome_v3'

In [5]:
### Off-line steps:

# Generation of core metabolome database.  See:
'http://localhost:8888/notebooks/PycharmProjects/core_metabolome/core_metabolome_db.ipynb'
core_metabolome = '/Users/dis/PycharmProjects/core_metabolome/core_metabolome_v3.pickle'

# Identification of good quality datasets.
'http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/high_quality_data_investigations.ipynb'
good_ds_ids = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/good_ds_2020_Feb_25.txt'

# Good datasets were researched from the beta server.  Run 1x w/ "reprocess = True" and
# "reprocess_not_downloading = True".  Run again w/False.  See:
'http://localhost:8888/notebooks/PycharmProjects/neutral_loss/good_nl_reports/Beta_server_neutral_losses_mass_search.ipynb'
beta_variables = {'NEUTRAL_LOSSES': ['-H2O'], 'MAX_FDR': 0.5, 'molDBs': 'core_metabolome_v3'}
path_to_reports = 'tbd'

In [6]:
# nl_00: Generates FP4 descriptors ('bits') and Mordred descriptors. 
# 777 seconds to reprocess everything on 11k. 22s without reprocessing.

db_out_0 = 'databases/core_metabolome_out.pickle'
db_in_0 = core_metabolome
dfs = main_loop_0(db_out_0, db_in_0, True, False)

db_df = dfs[0]
bits_df = dfs[1]
mord_norm_df = dfs[2]

Elapsed time:

20.17292070388794

Executed without error

databases/core_metabolome_out.pickle


In [15]:
db_df.to_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/core_metabolome_out.pickle') # 0.1 Gb
bits_df.to_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/bits_df.pickle')  #0.2 Gb
mord_norm_df.to_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/mord_norm_df.pickle')  # 0.8 Gb

In [7]:
### Wait for runs to complete on beta server!

In [9]:
# Merge pickles for each dataset.
path_to_reports = '/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/good_nl_reports_core_metabolome_v3'
master_df = pd.DataFrame()
counter = 0

for root, dirs, files in os.walk(path_to_reports):
    for file in files:
        if file.endswith(".pickle"):
            if counter % 50 == 0:
                print(counter)
            counter +=1
            current_df = pd.read_pickle(root + '/' + file)
            master_df = pd.concat([master_df, current_df], sort=True)

0
50
100
150
200
250
300


In [12]:
# Water only.  Edit for cases where neutral losses other than water under investigation.
terse_headers_0 = ['formula', 'adduct', 'ds_id', 'has_no_loss', 'has_H2O',
                   'msm', 'fdr', 'off_sample', 'hmdb_ids', 'intensity_avg',
                   'msm_H2O', 'fdr_H2O', 'off_sample_H2O', 'intensity_avg_H2O',
                   'colocalization_H2O', 'loss_intensity_ratio_H2O', 'loss_intensity_share_H2O',
                  'ion', 'ion_H2O', 'ion_formula', 'ion_formula_H2O']

master_df = master_df[terse_headers_0].copy(deep=True)
ds_id_meta = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/good_nl_reports/ds_id_meta.pickle')
master_df = master_df.merge(ds_id_meta, left_on='ds_id', right_on= 'datasetId', how='left')
master_df.drop(columns=['datasetId']).to_pickle('high_quality_cm3.pickle')

Exploratory data analysis experimental space:
http://localhost:8888/notebooks/PycharmProjects/neutral_loss/Exploratory_data_analysis.ipynb

Exploraatory data analysis chemical space:
http://localhost:8888/notebooks/PycharmProjects/neutral_loss/Exploratory_data_analysis_chemical_space.ipynb#

In [6]:
# Loads pickles from previous step
# For QC, don't need to run for reprocess.
input_df = pd.read_pickle('high_quality_cm3.pickle')
hmdb_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/core_metabolome_out.pickle') # 0.1 Gb
mord_norm_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/mord_norm_df.pickle')  # 0.8 Gb
bits_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/bits_df.pickle')  #0.2 Gb

In [5]:
### nl_01 to nl_03 need to be updated after exploratory data analysis is finished!!!

In [15]:
# nl_01: Parses METASPACE output and filters HMDB for observed
%run nl_01_preprocess.py -m high_quality_cm3.pickle -p /Users/dis/PycharmProjects/neutral_loss/databases/core_metabolome_out.pickle -is_H2O True

Elapsed time (seconds):

3

Executed without error

high_quality_cm3_output_01.pickle
high_quality_cm3_hmdb_01.pickle


In [19]:
# Loads pickles from previous step, 3 seconds
# For QC, don't need to run for reprocess.
output_1 = pd.read_pickle('high_quality_cm3_output_01.pickle')
hmdb_1 = pd.read_pickle('high_quality_cm3_hmdb_01.pickle')

In [21]:
# nl_02: Joins METASPACE output and observed HMDB, 40 seconds
%run nl_02_join.py --o high_quality_cm3_output_01.pickle --h high_quality_cm3_hmdb_01.pickle

Elapsed time:


Executed without error

39.665668964385986
high_quality_cm3_output_02.pickle


In [22]:
# Load X's if running machine learning
join_df_path = 'high_quality_cm3_output_02.pickle' 
join_df = pd.read_pickle(join_df_path)  # 0.5 Gb
unique_hmdbs = list(join_df.hmdb_ids.unique())

mord_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/mord_norm_df.pickle')
mord_df = mord_df[mord_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

bits_df = pd.read_pickle('/Users/dis/PycharmProjects/neutral_loss/databases/bits_df.pickle')  
bits_df = bits_df[bits_df.hmdb_ids.isin(unique_hmdbs)].copy(deep=True)

In [25]:
join_df.analyzer.unique()

array(['FTICR', 'Orbitrap'], dtype=object)

In [26]:
# Can use this to filter on inferred vaccuum maldi vs ap maldi
all_ds = list(join_df.ds_id.unique())
ap_ds = list(join_df[join_df.analyzer == 'FTICR'].ds_id.unique())
vac_ds = list(join_df[join_df.analyzer != 'FTICR'].ds_id.unique())

In [30]:
# nl_03: Filtering and machine learning models
datasets = ['all_ds', 'ap_ds', 'vac_ds']
ds_dict = {'all_ds': all_ds, 'ap_ds': ap_ds, 'vac_ds': vac_ds}
dataset = [datasets[0]]
target = ['H2O'] # ['H2O']
polarity = [1] # {1:positive, -1:negative] 
fdrs = [0.20, 0.10, 0.05] # [0.20, 0.10, 0.05]
colocalizations = [0.00, 0.50, 0.75] # [0.00, 0.50, 0.75]
min_n_obs = [1, 4, 10] # [1, 4, 10]
one_id_only = [False] # [True, False]

# Outputs (y), and weights (w):
global_ys = ['n_loss_only_H2O', 'n_loss_wparent_H2O', 'n_loss_all_H2O']
y = ['n_loss_wparent_H2O']
w = ['weight']
w_norms = [False, '10_y_bins', 'n_obs', 'isobar',
          '10_y_bins_W_n_obs', '10_y_bins_W_isobar', 'n_obs_W_isobar',
          '10_y_bins_W_n_obs_W_isobar']
w_norm = [False, '10_y_bins', 'n_obs', 'isobar',
          '10_y_bins_W_n_obs', '10_y_bins_W_isobar', 'n_obs_W_isobar',
          '10_y_bins_W_n_obs_W_isobar']

# Class of model:
models = ['direct', 'ml', 'deepchem']
model = [models[1]]

# Split.  If true, any formula only appears in train/test/validate.
single_fold_group = True

filter_model_df = pd.DataFrame(columns=['target', 'polarity', 'fdr', 'coloc', 'min_n_obs'
                                       'model', 'submodel', 'params', 'one_id_only',
                                        'X', 'y', 'w'])
# Specific inputs (X) for each model:
if model[0] is 'direct':
    submodel = ['']
    params = ['']
    Xs = ['trues', 'falses', 'rando', 'H2O_Present', 'n_loss_wparent_H2O']
    X = Xs                           
elif model[0] is 'ml':
    submodels = ['random_forest', 'XGBoost', 'SVM']
    submodel = [submodels[1]]
    params = ['']
    Xs = ['mord_norm', 'bits'] 
    X = ['mord_norm', 'bits']
elif model[0] is 'deepchem':
    submodels = [('GraphConvModel', 'GraphConv'), 
              ('WeaveModel', 'Weave'), 
              ('MPNNModel','Weave') ] 
    submodel = [submodels[0]]    
    params = ['']
    Xs = ['Molecule', 'Smiles']
    X = [Xs[0]]          
    
model_list = list(itertools.product(*[target, polarity, fdrs, colocalizations, min_n_obs,
                                      model, submodel, params, one_id_only,
                                      X, y, w, w_norm, dataset]))
print()
filter_param_df = pd.DataFrame(model_list, columns=['target', 'polarity', 'fdr', 'coloc', 'min_n_obs',
                                                    'model', 'submodel', 'params', 'one_id_only',
                                                    'X', 'y', 'w', 'w_norm', 'dataset'])

filter_param_df

,target,polarity,fdr,coloc,min_n_obs,model,submodel,params,one_id_only,X,y,w,w_norm,dataset
0,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,n_loss_wparent_H2O,weight,False,all_ds
1,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,n_loss_wparent_H2O,weight,10_y_bins,all_ds
2,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,n_loss_wparent_H2O,weight,n_obs,all_ds
3,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,n_loss_wparent_H2O,weight,isobar,all_ds
4,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,n_loss_wparent_H2O,weight,10_y_bins_W_n_obs,all_ds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,n_loss_wparent_H2O,weight,isobar,all_ds
428,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,n_loss_wparent_H2O,weight,10_y_bins_W_n_obs,all_ds
429,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,n_loss_wparent_H2O,weight,10_y_bins_W_isobar,all_ds
430,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,n_loss_wparent_H2O,weight,n_obs_W_isobar,all_ds


In [31]:
model_results = filter_split_model_score(filter_param_df, join_df_path, mord_df, bits_df,
                                         single_fold_group, ds_dict[dataset[0]])

start 0
filtering 0
aggregating 0
checking aggregation: 
             hmdb_ids     y  n_obs
12790   HMDB0049676+K   0.0      1
16575   HMDB0114925+K   0.0     45
3163    HMDB0007861+H   3.0      5
5708   HMDB0008995+Na  30.0     43
7252    HMDB0010357+H   1.0      8
(17685, 7)
Printing histogram!
n_obs_plots/2020_04_06_14_40_38_960098
Printing histogram!
hist_plots/2020_04_06_14_40_39_132751
joining X 0
['hmdb_ids', 'formula', 'intensity_avg', 'intensity_nl', 'y', 'w', 'n_obs']
Printing intensities parent and NL!
int_plots/2020_04_06_14_40_39_930605
splitting 0


/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/numpy/lib/histograms.py:829: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/numpy/lib/histograms.py:830: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


X_df_shapes:
(8692, 1826)
(4115, 1826)
Testing split, n overlap train/test: 
{'formula': 0, 'hmdb_ids': 0, 'y': 70, 'X': 3}
modeling 0
ml :  XGBoost
train_y: (8692,)
train_w: (8692,)
train_X: (8692, 1826)
Weighting:  False
Printing regression actual versus predicted output!
model_plots/2020_04_06_14_42_31_150253
start 1
filtering 1
aggregating 1
checking aggregation: 
             hmdb_ids     y  n_obs
1477    HMDB0003229+H   1.0      9
663     HMDB0000706+K   0.0      1
7333    HMDB0010400+H   1.0     43
10796  HMDB0042589+Na   0.0      3
8584    HMDB0011511+H  12.0     49
(17685, 7)
Printing histogram!
n_obs_plots/2020_04_06_14_42_33_639800
Printing histogram!
hist_plots/2020_04_06_14_42_33_809770
joining X 1
['hmdb_ids', 'formula', 'intensity_avg', 'intensity_nl', 'y', 'w', 'n_obs']
Printing intensities parent and NL!
int_plots/2020_04_06_14_42_34_525034
splitting 1


/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/numpy/lib/histograms.py:829: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/Users/dis/anaconda3/envs/lcenv/lib/python3.6/site-packages/numpy/lib/histograms.py:830: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


X_df_shapes:
(8452, 1826)
(4564, 1826)
Testing split, n overlap train/test: 
{'formula': 0, 'hmdb_ids': 0, 'y': 68, 'X': 4}
modeling 1
ml :  XGBoost
train_y: (8452,)
train_w: (8452,)
train_X: (8452, 1826)
Weighting:  10_y_bins
Printing regression actual versus predicted output!
model_plots/2020_04_06_14_44_19_667845
start 2
filtering 2
aggregating 2
checking aggregation: 
             hmdb_ids     y  n_obs
12015  HMDB0046099+Na   0.0      5
11142   HMDB0043854+H   0.0     66
4832    HMDB0008518+H  13.0     21
15019   HMDB0062758+K   0.0      2
16836  HMDB0115067+Na   0.0      3
(17685, 7)
Printing histogram!
n_obs_plots/2020_04_06_14_44_22_024027
Printing histogram!
hist_plots/2020_04_06_14_44_22_192663
joining X 2
['hmdb_ids', 'formula', 'intensity_avg', 'intensity_nl', 'y', 'w', 'n_obs']
Printing intensities parent and NL!
int_plots/2020_04_06_14_44_22_930625
splitting 2


...

Printing regression actual versus predicted output!
model_plots/2020_04_06_20_07_28_023343
start 430
filtering 430
aggregating 430
checking aggregation: 
            hmdb_ids    y  n_obs
5973   HMDB0044662+K  0.0      1
1722   HMDB0008131+H  0.0      7
8135  HMDB0114986+Na  6.0     34
6664   HMDB0049843+K  0.0     13
1770   HMDB0008148+H  3.0     92
(8641, 7)
Printing histogram!
n_obs_plots/2020_04_06_20_07_29_971404
Printing histogram!
hist_plots/2020_04_06_20_07_30_143730
joining X 430
['hmdb_ids', 'formula', 'intensity_avg', 'intensity_nl', 'y', 'w', 'n_obs']
Printing intensities parent and NL!
int_plots/2020_04_06_20_07_30_401984
splitting 430
X_df_shapes:
(1476, 1024)
(608, 1024)
Testing split, n overlap train/test: 
{'formula': 0, 'hmdb_ids': 0, 'y': 5, 'X': 28184}
modeling 430
ml :  XGBoost
train_y: (1476,)
train_w: (1476,)
train_X: (1476, 1024)
Weighting:  n_obs_W_isobar


/Users/dis/PycharmProjects/neutral_loss/nl_03_filter_model_score_reg.py:482: RuntimeWarning: divide by zero encountered in true_divide
  weights = 1 / density


Printing regression actual versus predicted output!
model_plots/2020_04_06_20_07_40_253309
start 431
filtering 431
aggregating 431
checking aggregation: 
            hmdb_ids    y  n_obs
1581   HMDB0008080+K  0.0     79
3741   HMDB0009547+H  0.0     34
2987   HMDB0008951+H  0.0      1
3779   HMDB0009580+K  0.0     10
5290  HMDB0029109+Na  0.0      3
(8641, 7)
Printing histogram!
n_obs_plots/2020_04_06_20_07_42_251860
Printing histogram!
hist_plots/2020_04_06_20_07_42_425997
joining X 431
['hmdb_ids', 'formula', 'intensity_avg', 'intensity_nl', 'y', 'w', 'n_obs']
Printing intensities parent and NL!
int_plots/2020_04_06_20_07_42_674818
splitting 431
X_df_shapes:
(1708, 1024)
(902, 1024)
Testing split, n overlap train/test: 
{'formula': 0, 'hmdb_ids': 0, 'y': 9, 'X': 46775}
modeling 431
ml :  XGBoost
train_y: (1708,)
train_w: (1708,)
train_X: (1708, 1024)
Weighting:  10_y_bins_W_n_obs_W_isobar


/Users/dis/PycharmProjects/neutral_loss/nl_03_filter_model_score_reg.py:482: RuntimeWarning: divide by zero encountered in true_divide
  weights = 1 / density


Printing regression actual versus predicted output!
model_plots/2020_04_06_20_07_53_818586
Elapsed time:


Executed without error

13.566563844680786


# To do:
1. Add M and M.
2. nl_03 code review.
3. Generate +/- results.
4. Discuss and predict classifier development.
5. Find reasonable result and split by instrument/lab

In [32]:
model_results.round(3)

,target,polarity,fdr,coloc,min_n_obs,model,submodel,params,one_id_only,X,...,dataset,rmse_train,rmse_test,r2_test,rmse_train_w,rmse_test_w,r2_test_w,n_test,plt_path,time
0,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.213,0.273,0.130,NaN,NaN,NaN,4115,model_plots/2020_04_06_14_42_31_150253,114
1,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.271,0.325,-0.127,0.173,0.304,-0.025,4564,model_plots/2020_04_06_14_44_19_667845,108
2,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.232,0.272,0.140,0.183,0.225,0.339,4358,model_plots/2020_04_06_14_46_16_983509,117
3,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.208,0.281,0.208,0.213,0.310,0.119,4023,model_plots/2020_04_06_14_48_16_657519,119
4,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.268,0.306,-0.010,0.186,0.297,-0.094,4070,model_plots/2020_04_06_14_50_15_009077,118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,...,all_ds,0.170,0.183,-0.009,0.173,0.224,-0.066,932,model_plots/2020_04_06_20_07_03_912409,13
428,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,...,all_ds,0.203,0.303,-0.441,0.200,0.374,-0.404,744,model_plots/2020_04_06_20_07_16_443494,12
429,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,...,all_ds,0.302,0.290,-0.669,0.158,0.418,-0.396,1014,model_plots/2020_04_06_20_07_28_023343,11
430,H2O,1,0.05,0.75,10,ml,XGBoost,,False,bits,...,all_ds,0.196,0.271,0.045,0.173,0.254,-0.026,608,model_plots/2020_04_06_20_07_40_253309,12


In [40]:
model_results.sort_values(by=['r2_test_w'], ascending=False).round(3)

,target,polarity,fdr,coloc,min_n_obs,model,submodel,params,one_id_only,X,...,dataset,rmse_train,rmse_test,r2_test,rmse_train_w,rmse_test_w,r2_test_w,n_test,plt_path,time
18,H2O,1,0.20,0.00,4,ml,XGBoost,,False,mord_norm,...,all_ds,0.209,0.252,0.259,0.192,0.235,0.347,2760,model_plots/2020_04_06_15_07_04_411765,69
2,H2O,1,0.20,0.00,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.232,0.272,0.140,0.183,0.225,0.339,4358,model_plots/2020_04_06_14_46_16_983509,117
290,H2O,1,0.05,0.00,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.212,0.290,0.133,0.185,0.247,0.325,2402,model_plots/2020_04_06_18_58_35_638956,62
226,H2O,1,0.10,0.50,10,ml,XGBoost,,False,mord_norm,...,all_ds,0.165,0.175,0.301,0.157,0.205,0.297,1786,model_plots/2020_04_06_18_14_26_126220,36
338,H2O,1,0.05,0.50,1,ml,XGBoost,,False,mord_norm,...,all_ds,0.147,0.266,0.034,0.168,0.261,0.294,2283,model_plots/2020_04_06_19_24_09_665985,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,H2O,1,0.05,0.75,1,ml,XGBoost,,False,bits,...,all_ds,0.213,0.247,0.137,NaN,NaN,NaN,2031,model_plots/2020_04_06_19_52_59_724586,28
400,H2O,1,0.05,0.75,4,ml,XGBoost,,False,mord_norm,...,all_ds,0.147,0.227,0.081,NaN,NaN,NaN,1050,model_plots/2020_04_06_19_56_59_444359,34
408,H2O,1,0.05,0.75,4,ml,XGBoost,,False,bits,...,all_ds,0.190,0.183,-0.256,NaN,NaN,NaN,1179,model_plots/2020_04_06_20_01_06_120443,17
416,H2O,1,0.05,0.75,10,ml,XGBoost,,False,mord_norm,...,all_ds,0.133,0.268,0.085,NaN,NaN,NaN,763,model_plots/2020_04_06_20_03_38_651768,21


In [35]:
model_results.to_pickle('all_model_results.pickle')

In [34]:
# Regression results:
#all_model_results = pd.read_pickle('all_model_results.pickle')
#all_model_results = pd.concat([all_model_results, model_results])
all_model_results.to_pickle('all_model_results.pickle')
all_model_results.round(3)

NameError: name 'all_model_results' is not defined

In [ ]:
all_model_results.sort_values(by=['rmse_test']).round(3)

In [ ]:
temp = dict(all_model_results[(all_model_results.index == 90) & 
                  (all_model_results.X == 'mord_norm')].round(2))

In [ ]:
temp